In [1]:
# import libraries

import pandas as pd
import numpy as np
#import postgresql
import getpass
import psycopg2
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# creating postgrest connection

conn = psycopg2.connect(host="d-an-test.cygqmbt4gi1l.eu-central-1.rds.amazonaws.com",
                        user="bruna_miguel",
                       password=getpass.getpass(),
                       database="postgres")

········


Calling all the tables from the Database to work on Python

In [4]:
# fbt_conversions table

query_fbt_conversions = """
SELECT *
From fbt_conversions;
"""

fbt_conversions = pd.read_sql(query_fbt_conversions, conn)

In [5]:
fbt_conversions.head()

,sk_user,purchased_at,revenue,subscription_id
0,5734568301,2020-05-16 17:42:06,14.36,964462
1,5734568301,2020-05-16 17:42:06,14.36,964462
2,5734112101,2020-05-11 21:49:40,18.39,962593
3,5734112101,2020-05-11 21:49:40,18.39,962593
4,88254113,2020-11-30 11:48:19,25.60,1060301


In [7]:
# fbt_facebook_campaign table

query_fbt_facebook_campaigns = """
SELECT *
From fbt_facebook_campaigns;
"""

fbt_facebook_campaigns = pd.read_sql(query_fbt_facebook_campaigns, conn)

In [8]:
fbt_facebook_campaigns.head()

,campaign_name,campaign_id,spend_date,impressions,spend
0,android_campaign_world,339703,2020-11-17,783408,1852.0
1,ios_campaign_north_america,379581,2020-11-17,305051,3593.0
2,web_campaign_world,280020,2020-11-17,440302,2079.0
3,ios_campaign_world,279755,2020-11-17,2128655,12962.0
4,web_campaign_world,280020,2020-11-16,516010,1784.0


In [10]:
# fbt_installs table

query_fbt_installs = """
SELECT *
From fbt_installs;
"""

fbt_installs = pd.read_sql(query_fbt_installs, conn)

In [11]:
fbt_installs.head()

,sk_user,install_timestamp,network,campaign_id
0,5761711975,2020-12-08 07:23:56,Facebook & Instagram,280020
1,5733117048,2021-01-02 19:41:15,Facebook & Instagram,280020
2,5760786737,2021-01-02 15:13:09,Apple Search Ads,
3,5743168440,2020-11-09 01:30:04,Facebook & Instagram,379581
4,5751881847,2020-09-13 14:44:34,Asana Rebel Website,


In [12]:
# fbt_web_sessions table

query_fbt_web_sessions = """
SELECT *
From fbt_web_sessions;
"""

fbt_web_sessions = pd.read_sql(query_fbt_web_sessions, conn)

In [13]:
fbt_web_sessions.head()

,sk_web_user,session_id,session_started_timestamp,network,campaign_id
0,8200456,bf875798-f11d-4103-b2d8-8bb2487acea6,2020-12-08 12:12:56,facebook,280020
1,8204683,0941cd1a-503e-4101-a48f-5a27fb88f8bf,2020-12-08 11:44:16,facebook,280020
2,8197168,dc022bf3-cae3-485d-b450-5913847d710b,2020-12-08 14:29:30,facebook,280020
3,8221759,56027fbd-8346-43e8-906b-4fc3d1f3d7f4,2020-12-08 08:56:47,facebook,280020
4,8208007,41a8642e-3faf-41eb-96db-1185128a9a96,2020-12-08 06:57:20,facebook,280020


In [14]:
# fbt_web_user_mapping

query_fbt_web_user_mapping = """
SELECT *
From fbt_web_user_mapping;
"""

fbt_web_user_mapping = pd.read_sql(query_fbt_web_user_mapping, conn)

In [15]:
fbt_web_user_mapping.head()

,sk_user,domain_userid,sk_web_user
0,5751811765,863cc984-e59f-4e79-9f51-1821be329690,7201041
1,5727750686,0ccb73e5-f1e8-4761-af7a-839ff66899db,7766318
2,5763935126,94c15d51-d92c-4928-a62f-3835d890b951,8554652
3,5763459124,acb9b264-7b25-4325-8574-0d1d385e4c6c,8413852
4,4748728864,6b54c319-93af-4afc-b5b5-1fb63df37fdf,7875548


# Analysis

In [19]:
#confirm the 5 campaigns

fbt_facebook_campaigns['campaign_name'].value_counts()

ios_campaign_world            30
android_campaign_world        30
ios_campaign_north_america    27
web_campaign_world            18
web_campaign_world_promo       5
Name: campaign_name, dtype: int64

The web campaigns web_campaign_world and wed_campaign_world_promo have a different behaviour. 
Analysing app campaigns first

## App Campaigns

In [22]:
fbt_facebook_campaigns.head()

,campaign_name,campaign_id,spend_date,impressions,spend
0,android_campaign_world,339703,2020-11-17,783408,1852.0
1,ios_campaign_north_america,379581,2020-11-17,305051,3593.0
2,web_campaign_world,280020,2020-11-17,440302,2079.0
3,ios_campaign_world,279755,2020-11-17,2128655,12962.0
4,web_campaign_world,280020,2020-11-16,516010,1784.0


In [21]:
fbt_installs.head()

,sk_user,install_timestamp,network,campaign_id
0,5761711975,2020-12-08 07:23:56,Facebook & Instagram,280020
1,5733117048,2021-01-02 19:41:15,Facebook & Instagram,280020
2,5760786737,2021-01-02 15:13:09,Apple Search Ads,
3,5743168440,2020-11-09 01:30:04,Facebook & Instagram,379581
4,5751881847,2020-09-13 14:44:34,Asana Rebel Website,


In [24]:
# merging tables in order to have the revenue and spend of app campaign together

app_spend = fbt_facebook_campaigns.merge(fbt_installs, how="left", left_on="campaign_id", right_on="campaign_id")

app_spend.head()

,campaign_name,campaign_id,spend_date,impressions,spend,sk_user,install_timestamp,network
0,android_campaign_world,339703,2020-11-17,783408,1852.0,5751881627,2020-10-23 06:34:37,Facebook & Instagram
1,android_campaign_world,339703,2020-11-17,783408,1852.0,5757410977,2020-11-02 08:07:36,Facebook & Instagram
2,android_campaign_world,339703,2020-11-17,783408,1852.0,84627366,2020-11-08 13:48:57,Facebook & Instagram
3,android_campaign_world,339703,2020-11-17,783408,1852.0,5755009149,2020-10-13 12:08:27,Facebook & Instagram
4,android_campaign_world,339703,2020-11-17,783408,1852.0,5745019978,2020-10-13 10:55:34,Facebook & Instagram


In [27]:
app_spend_revenue = app_spend.merge(fbt_conversions, how="left", left_on="sk_user", right_on="sk_user")
app_spend_revenue.head()

,campaign_name,campaign_id,spend_date,impressions,spend,sk_user,install_timestamp,network,purchased_at,revenue,subscription_id
0,android_campaign_world,339703,2020-11-17,783408,1852.0,5751881627,2020-10-23 06:34:37,Facebook & Instagram,NaT,NaN,NaN
1,android_campaign_world,339703,2020-11-17,783408,1852.0,5757410977,2020-11-02 08:07:36,Facebook & Instagram,NaT,NaN,NaN
2,android_campaign_world,339703,2020-11-17,783408,1852.0,84627366,2020-11-08 13:48:57,Facebook & Instagram,NaT,NaN,NaN
3,android_campaign_world,339703,2020-11-17,783408,1852.0,5755009149,2020-10-13 12:08:27,Facebook & Instagram,NaT,NaN,NaN
4,android_campaign_world,339703,2020-11-17,783408,1852.0,5745019978,2020-10-13 10:55:34,Facebook & Instagram,NaT,NaN,NaN


In [29]:
app_spend_revenue["campaign_name"].value_counts()

android_campaign_world        7429980
ios_campaign_world            6059670
web_campaign_world             338346
ios_campaign_north_america     230904
web_campaign_world_promo         9225
Name: campaign_name, dtype: int64

In [34]:
#select only app campaigns

app_campaign_df=app_spend_revenue.loc[(app_spend_revenue["campaign_name"]=="android_campaign_world") | (app_spend_revenue["campaign_name"]=="ios_campaign_world") | (app_spend_revenue["campaign_name"]=="ios_campaign_north_america")]

In [35]:
app_campaign_df.head()

,campaign_name,campaign_id,spend_date,impressions,spend,sk_user,install_timestamp,network,purchased_at,revenue,subscription_id
0,android_campaign_world,339703,2020-11-17,783408,1852.0,5751881627,2020-10-23 06:34:37,Facebook & Instagram,NaT,NaN,NaN
1,android_campaign_world,339703,2020-11-17,783408,1852.0,5757410977,2020-11-02 08:07:36,Facebook & Instagram,NaT,NaN,NaN
2,android_campaign_world,339703,2020-11-17,783408,1852.0,84627366,2020-11-08 13:48:57,Facebook & Instagram,NaT,NaN,NaN
3,android_campaign_world,339703,2020-11-17,783408,1852.0,5755009149,2020-10-13 12:08:27,Facebook & Instagram,NaT,NaN,NaN
4,android_campaign_world,339703,2020-11-17,783408,1852.0,5745019978,2020-10-13 10:55:34,Facebook & Instagram,NaT,NaN,NaN


In [36]:
# confirm

app_campaign_df["campaign_name"].value_counts()

android_campaign_world        7429980
ios_campaign_world            6059670
ios_campaign_north_america     230904
Name: campaign_name, dtype: int64